# ਸੈਸ਼ਨ 6 – ਬਹੁ-ਪੜਾਅ ਪਾਈਪਲਾਈਨ (ਯੋਜਨਾ → ਕਾਰਵਾਈ → ਸੁਧਾਰ)

ਰੂਟਿੰਗ ਫੰਕਸ਼ਨਾਂ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਹਰ ਪੜਾਅ ਲਈ ਮਾਡਲ ਚੁਣਦਾ ਹੈ, ਫਿਰ ਅੰਤਮ ਜਵਾਬ ਨੂੰ ਸੁਧਾਰਦਾ ਹੈ।


### ਵਿਆਖਿਆ: Dependency Installation  
`foundry-local-sdk` ਅਤੇ `openai` ਨੂੰ ਇੰਸਟਾਲ ਕਰਦਾ ਹੈ ਜੋ ਪ੍ਰਤੀ-ਕਦਮ ਚੈਟ ਕਾਲਾਂ ਲਈ ਲੋੜੀਂਦੇ ਹਨ। ਦੁਬਾਰਾ ਚਲਾਉਣਾ ਸੁਰੱਖਿਅਤ ਹੈ।  


# ਸਥਿਤੀ
ਬਹੁ-ਪੜਾਅ ਮਾਡਲ ਪਾਈਪਲਾਈਨ ਜੋ: (1) ਕੰਮ ਨੂੰ ਵੱਖ-ਵੱਖ ਪੜਾਅ ਵਿੱਚ ਯੋਜਿਤ ਕਰਦਾ ਹੈ, (2) ਹਰ ਪੜਾਅ ਨੂੰ ਇਰਾਦਾ-ਅਧਾਰਿਤ ਮਾਡਲ ਚੋਣ ਨਾਲ ਅੰਜਾਮ ਦਿੰਦਾ ਹੈ, (3) ਅੰਤਮ ਤੌਰ 'ਤੇ ਸੰਕਲਿਤ ਜਵਾਬ ਨੂੰ ਸੁਧਾਰਦਾ ਹੈ। ਕੁਸ਼ਲਤਾ ਲਈ ਵੱਖ-ਵੱਖ SLM ਸਮਰਥਨ ਨੂੰ ਜੋੜਦੇ ਹੋਏ ਗੁਣਵੱਤਾ ਨੂੰ ਬਰਕਰਾਰ ਰੱਖਣ ਦਾ ਪ੍ਰਦਰਸ਼ਨ।


In [13]:
!pip install -q foundry-local-sdk openai

### ਵਿਆਖਿਆ: ਮੁੱਖ ਇੰਪੋਰਟਸ  
ਇੰਟੈਂਟ ਡਿਟੈਕਸ਼ਨ ਲਈ ਰੇਗੁਲਰ ਐਕਸਪ੍ਰੈਸ਼ਨ (regex) ਨੂੰ ਇੰਪੋਰਟ ਕਰਦਾ ਹੈ, ਪ੍ਰਤੀ-ਅਲਿਆਸ ਅਟੈਚਮੈਂਟ ਲਈ Foundry Local ਮੈਨੇਜਰ ਨੂੰ, ਅਤੇ ਚੈਟ ਕਮਪਲੀਸ਼ਨ ਲਈ OpenAI ਕਲਾਇੰਟ ਨੂੰ।


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### ਵਿਆਖਿਆ: ਸਮਰੱਥਾ ਕੈਟਾਲਾਗ ਅਤੇ ਨਿਯਮ  
ਇਹ ਸਮਰੱਥਾ-ਜਾਣੂ ਕੈਟਾਲਾਗ ਅਤੇ ਰੇਗੁਲਰ ਐਕਸਪ੍ਰੈਸ਼ਨ ਨਿਯਮਾਂ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਦਾ ਹੈ ਜੋ ਕਦਮ ਦੇ ਪਾਠ ਨੂੰ ਇਰਾਦੇ ਦੀਆਂ ਸ਼੍ਰੇਣੀਆਂ (ਕੋਡ, ਸੰਖੇਪ, ਵਰਗੀਕਰਨ, ਜਨਰਲ) ਨਾਲ ਜੋੜਨ ਲਈ ਵਰਤੇ ਜਾਂਦੇ ਹਨ। ਜਦੋਂ ਕਈ ਮਾਡਲ ਇੱਕੋ ਇਰਾਦੇ ਦਾ ਸਮਰਥਨ ਕਰਦੇ ਹਨ, ਤਾਂ ਘੱਟ ਤਰਜੀਹ ਵਾਲੀਆਂ ਮੁੱਲਾਂ ਨੂੰ ਤਰਜੀਹ ਮਿਲਦੀ ਹੈ।


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### ਵਿਆਖਿਆ: ਇਰਾਦਾ, ਮਾਡਲ ਚੋਣ ਅਤੇ ਚੈਟ ਸਹਾਇਕ
ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ:
- `detect_intent` ਰੇਗੁਲਰ ਐਕਸਪ੍ਰੈਸ਼ਨ-ਅਧਾਰਿਤ ਵਰਗੀਕਰਨ ਲਈ।
- `pick_model` ਯੋਗਤਾ + ਤਰਜੀਹ ਦੇ ਆਧਾਰ 'ਤੇ ਸਭ ਤੋਂ ਵਧੀਆ ਉਪਨਾਮ ਚੁਣਨ ਲਈ।
- `chat` ਸੁਵਿਧਾਜਨਕ ਵ੍ਰੈਪਰ ਜੋ ਹਰ ਉਪਨਾਮ ਲਈ ਇੱਕ ਕਲਾਇੰਟ ਬਣਾਉਂਦਾ ਹੈ ਅਤੇ ਇੱਕ-ਟਰਨ ਜਵਾਬ ਵਾਪਸ ਕਰਦਾ ਹੈ।


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### ਵਿਆਖਿਆ: ਮਲਟੀ-ਸਟੈਪ ਪਾਈਪਲਾਈਨ ਫੰਕਸ਼ਨ
ਪਾਈਪਲਾਈਨ ਨੂੰ ਲਾਗੂ ਕਰਦਾ ਹੈ: ਯੋਜਨਾ → ਕਦਮਾਂ ਨੂੰ ਪਾਰਸ ਕਰੋ → ਹਰ ਇੱਕ ਨੂੰ ਇਰਾਦਾ-ਅਧਾਰਿਤ ਰੂਟਿੰਗ ਨਾਲ ਚਲਾਓ → ਮਿਲੇ-जੁਲੇ ਨਤੀਜਿਆਂ ਨੂੰ ਸੁਧਾਰੋ। ਜਾਂਚ ਜਾਂ ਮੁਲਾਂਕਨ ਲਈ ਸੰਰਚਿਤ dict ਵਾਪਸ ਕਰਦਾ ਹੈ।


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### ਵਿਆਖਿਆ: ਉਦਾਹਰਨ ਟਾਸਕ ਚਲਾਓ
ਪੂਰੀ ਪਾਈਪਲਾਈਨ ਨੂੰ ਇੱਕ ਰੀਫੈਕਟਰ-ਕੇਂਦਰਿਤ ਟਾਸਕ 'ਤੇ ਚਲਾਉਂਦਾ ਹੈ ਜੋ ਮਿਲੇ-ਜੁਲੇ ਇਰਾਦਿਆਂ (ਕੋਡ + ਸੰਖੇਪ) ਨੂੰ ਦਰਸਾਉਂਦਾ ਹੈ। ਨਤੀਜੇ ਦਾ ਡਿਕਟ ਕੱਚੇ ਯੋਜਨਾ, ਪ੍ਰਤੀ-ਕਦਮ ਨਤੀਜਿਆਂ ਨੂੰ ਚੁਣੇ ਮਾਡਲ ਅਲੀਅਸਾਂ ਨਾਲ, ਅਤੇ ਅੰਤਮ ਸੰਕਲਿਤ ਜਵਾਬ ਨੂੰ ਦਿਖਾਉਂਦਾ ਹੈ।


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### ਵਿਆਖਿਆ: ਨਤੀਜੇ ਦੀ ਵਸਤੂ ਦਿਖਾਓ  
ਤੇਜ਼ੀ ਨਾਲ ਜਾਂਚ ਕਰਨ ਜਾਂ ਅਗਲੇ ਮੁਲਾਂਕਣ ਲਈ ਸੰਰਚਿਤ ਪਾਈਪਲਾਈਨ ਆਉਟਪੁੱਟ ਦਿਖਾਉਂਦਾ ਹੈ (ਜਿਵੇਂ ਕਿ ਕਦਮ ਦੀ ਗੁਣਵੱਤਾ ਜਾਂ ਸੁਧਾਰ ਦੀ ਪ੍ਰਭਾਵਸ਼ੀਲਤਾ ਨੂੰ ਮਾਪਣਾ)।



---

**ਅਸਵੀਕਰਤੀਕਰਨ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚਨਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਅਸੀਂ ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
